In [15]:
import numpy as np
import cv2
from scipy.optimize import leastsq
import matplotlib.pyplot as plt
import glob
from ReadCameraModel import *
from UndistortImage import *
%matplotlib inline

# Data Preparation

camera_model = glob.glob('C:\\Users\\shant\\dataset\\model\\')
data_files = glob.glob('C:\\Users\\shant\\dataset\\stereo\\centre\\')

def intrinsic_matrix_undistorted_image(images,model_path):
    
    """
    Inputs:
    
    model_path: The path to the model files
    images: The input image
    
    Outputs:
    k_matrix: The intrinsic parameters
    undistorted_image: The undistorted image
    """
    image = cv2.imread(images,0)
    image = cv2.cvtColor(image, cv2.COLOR_BayerBG2BGR)
    
    fx, fy, cx, cy, G_camera_image, LUT = ReadCameraModel(model_path)
    k_matrix = np.zeros((3,3))
    k_matrix[0,0] = fx
    k_matrix[0,2] = cx
    k_matrix[1,1] = fy
    k_matrix[1,2] = cy
    k_matrix[2,2] = 1
    
    undistorted_image = UndistortImage(image, LUT)
    
    return (undistorted_image, k_matrix)

# Estimating the Fundamental and Essential Matrix

def FundamentalMatrix(input_points, output_correspondence, scaling):
    
    """
    Inputs:
    input_points: This is a Nx2 Matrix of (x,y) points
    output_correspondance: This is a Nx2 Matrix of (x',y') points
    scaling: The maximum of the input images width and height
    
    """
    # Normalize the input coordinates with the scaling factor
    
    pts1 = input_points / scaling
    pts2 = output_correspondence / scaling
    
    # List of Fundamental Matrix
    F_list = []
    
    # Transformation matrix for unnormalizing the fundamental matrix
    T = np.array([[1/scaling,0,0],[0,1/scaling,0],[0,0,1]])
    
    # Construct the A matrix 
    first_row_A = np.array([[pts1[:,0][0]*pts2[:,0][0], pts1[:,0][0]*pts2[:,1][0], pts1[:,0][0], pts1[:,1][0]*pts2[:,0][0], pts1[:,1][0]*pts2[:,1][0], pts1[:,1][0], pts2[:,0][0], pts2[:,1][0], 1]])
    second_row_A = np.array([[pts1[:,0][1]*pts2[:,0][1], pts1[:,0][1]*pts2[:,1][1], pts1[:,0][1], pts1[:,1][1]*pts2[:,0][1], pts1[:,1][1]*pts2[:,1][1], pts1[:,1][1], pts2[:,0][1], pts2[:,1][1], 1]])
    third_row_A = np.array([[pts1[:,0][2]*pts2[:,0][2], pts1[:,0][2]*pts2[:,1][2], pts1[:,0][2], pts1[:,1][2]*pts2[:,0][2], pts1[:,1][2]*pts2[:,1][2], pts1[:,1][2], pts2[:,0][2], pts2[:,1][2], 1]]) 
    fourth_row_A = np.array([[pts1[:,0][3]*pts2[:,0][3], pts1[:,0][3]*pts2[:,1][3], pts1[:,0][3], pts1[:,1][3]*pts2[:,0][3], pts1[:,1][3]*pts2[:,1][3], pts1[:,1][3], pts2[:,0][3], pts2[:,1][3], 1]])
    fifth_row_A = np.array([[pts1[:,0][4]*pts2[:,0][4], pts1[:,0][4]*pts2[:,1][4], pts1[:,0][4], pts1[:,1][4]*pts2[:,0][4], pts1[:,1][4]*pts2[:,1][4], pts1[:,1][4], pts2[:,0][4], pts2[:,1][4], 1]])
    sixth_row_A = np.array([[pts1[:,0][5]*pts2[:,0][5], pts1[:,0][5]*pts2[:,1][5], pts1[:,0][5], pts1[:,1][5]*pts2[:,0][5], pts1[:,1][5]*pts2[:,1][5], pts1[:,1][5], pts2[:,0][5], pts2[:,1][5], 1]])
    seventh_row_A = np.array([[pts1[:,0][6]*pts2[:,0][6], pts1[:,0][6]*pts2[:,1][6], pts1[:,0][6], pts1[:,1][6]*pts2[:,0][6], pts1[:,1][6]*pts2[:,1][6], pts1[:,1][6], pts2[:,0][6], pts2[:,1][6], 1]]) 
    eighth_row_A = np.array([[pts1[:,0][7]*pts2[:,0][7], pts1[:,0][7]*pts2[:,1][7], pts1[:,0][7], pts1[:,1][7]*pts2[:,0][7], pts1[:,1][7]*pts2[:,1][7], pts1[:,1][7], pts2[:,0][7], pts2[:,1][7], 1]])
    
    # Stack the rows to create the A matrix
    A = np.vstack((first_row_A,second_row_A,third_row_A,fourth_row_A,fifth_row_A,sixth_row_A,seventh_row_A,eighth_row_A,np.ones(9)))
    # Singular Value Decomposition
    U, S, Vh = np.linalg.svd(A)
    V = Vh.T
    
    # Constructing the fundamental matrix by taking the last column of the V matrix as it corresponds to the nullspace eigenvector
    fundamental_matrix = V[:,-1]
    fundamental_matrix = fundamental_matrix.reshape(3,3)
    
    # Enforcing Rank 2 constraint
    U, sigma, Vh = np.linalg.svd(fundamental_matrix)
    sigma[2] = 0
    fundamental_matrix = np.matmul(U, np.matmul(np.diag(sigma), Vh))
    
    # Unnormalize the fundmental matrix
    fundamental_matrix = np.matmul(np.matmul(T.T, fundamental_matrix), T)
    
    F_list.append(fundamental_matrix)
    return F_list
    
def get_keypoints(image1, image2):
    
    """
    Inputs:
    
    image1: left image
    image2: right image
    
    Outputs:
    
    ptsLeft: point correspondences for left image
    ptsRight: point correspondences for right image
    """

    # use sift keypoint to get the points
    sift = cv2.ORB_create()
    kp1, des1 = sift.detectAndCompute(image1, None)
    kp2, des2 = sift.detectAndCompute(image2, None)
    des1 = np.asarray(des1, np.float32)
    des2 = np.asarray(des2, np.float32)
    flann = cv2.FlannBasedMatcher(dict(algorithm = 0, trees = 5), dict(checks = 50))
    matches = flann.knnMatch(des1, des2, k = 2)
    good = []
    ptsLeft = []
    ptsRight = []

    for i,(m,n) in enumerate(matches):
        if m.distance < 0.8*n.distance:
            good.append(m)
            ptsRight.append(kp2[m.trainIdx].pt)
            ptsLeft.append(kp1[m.queryIdx].pt)

    ptsLeft = np.int32(ptsLeft)
    ptsRight = np.int32(ptsRight)
    return (ptsLeft, ptsRight)

def FundamentalMatrixRansac(input_points, output_correspondence, scaling):
    
    """
    Inputs:
    input_points: This is a randomly sampled input point correspondence
    output_correspondance: This is the output point correspondence
    
    Outputs:
    fundamental_matrix: The refined fundamental matrix after performing RANSAC
    """
    # Convert the correspondences into homogenous coordinates
    pts1 = np.hstack(input_points, np.ones(input_points.shape[0],1))
    pts2 = np.hstack(output_correspondence, np.ones(output_correspondence.shape[0], 1))
    
    # Number of iterations
    iterations = 1000
    
    # The threshold error
    epsilon = 0.01
    
    # The indices with value less than the error
    best_indices = None
    
    # Best Fundamental Matrix
    best_fundamental_matrix = None
    
    # Best Inliers
    best_inliers = 0
    
    for i in range(iterations):
        
        rand_index = np.random.choice(input_points.shape[0], 8, False)
        F = FundamentalMatrix(input_points[rand_index], output_correspondence[rand_index],scaling)
        
        for fundamental_matrix in F:
            
            # Print a List of indices
            indices = np.where(np.abs(np.matmul(pts2, np.matmul(fundamental_matrix, pts.T))).diagonal() < epsilon)[0]
            
            if len(indices) > best_inliers:
                
                best_fundamental_matrix = fundamental_matrix
                best_indices = indices
                best_inliers = len(indices)

# Compute the Essential Matrix
def EssentialMatrix(fundamental_matrix, calibration_matrix):
    
    """
    Inputs:
    fundamental_matrix: The fundamental matrix that gives the epipolar line on which an input point must lie
    calibration_matrix: The K matrix responsible for projecting an object in camera coordinates to the image coordinate system
    
    Ouput:
    essential_matrix: The essential matrix giving the relation between the 2 image points
    """
    essential_matrix = np.matmul(calibration_matrix.T, np.matmul(fundamental_matrix,calibration_matrix))
    U, sigma, Vh = np.linalg.svd(essential_matrix)
    
    sigma[0] = 1
    sigma[1] = 1
    sigma[2] = 0
    
    essential_matrix = np.matmul(U, np.matmul(np.diag(sigma), Vh))
    
    return essential_matrix

# Get the Camera Poses(translation and rotation matrices)
def ExtractCameraPose(essential_matrix):
    
    """
    Inputs:
    essential_matrix: The Essential Matrix
    
    Outputs:
    Camera Poses
    """
    
    W = np.array([[0,-1,0],[1,0,0],[0,0,1]])
    
    # Perform the Singular Value Decomposition of the Essential Matrix
    U, sigma, Vh = np.linalg.svd(essential_matrix)
    
    # Defining the 4 Camera Poses (c1,r1), (c2,r2), (c3,r3), (c4,r4)
    c1 = U[:,2]
    c2 = -U[:,2]
    c3 = U[:,2]
    c4 = -U[:,2]
    r1 = np.dot(U, np.dot(W, Vh))
    r2 = np.dot(U, np.dot(W, Vh))
    r3 = np.dot(U, np.dot(np.transpose(W), Vh))
    r4 = np.dot(U, np.dot(np.transpose(W), Vh))
    
    if np.linalg.det(r1) < 0:
        c1 = -c1
        r1 = -r1
    if np.linalg.det(r2) < 0:
        c2 = -c2
        r2 = -r2
    if np.linalg.det(r3) < 0:
        c3 = -c3
        r3 = -r3
    if np.linalg.det(r4) < 0:
        c4 = -c4
        r4 = -r4
    
    # Reshape the translational matrices from 1x3 --> 3x1
    c1 = c1.reshape(-1,1)
    c2 = c2.reshape(-1,1)
    c3 = c3.reshape(-1,1)
    c4 = c4.reshape(-1,1)
    
    return [np.array(c1), np.array(c2), np.array(c3), np.array(c4)],[np.array(r1), np.array(r2), np.array(r3), np.array(r4)]

# Triangulation Check for Cheirality 
def LinearTriangulation(K, C1, R1, C2, R2, pts1, pts2):
    
    """
    Inputs:
    K: Camera Calibration Matrix
    C1: The camera center for the first camera (3x1)
    C2: The camera center for the second camera (3x1)
    R1: The rotation matrix for the first camera (3x3)
    R2: The rotation matrix for the second camera (3x3)
    pts1: The points in left image whose correspondance needs to be found in the right image. (Nx2)
    pts2: The corresponding points in the right image (Nx2)
    
    Outputs:
    P: x, y, z world coordinates (Nx3)/ Triangulated points
    reprojection_error: The reprojection error from the world frame to the image frame
    """
    
    camera_pose1 = np.dot(np.dot(K,R1), np.concatenate((np.eye(3), -C1), axis = 1))
    camera_pose2 = np.dot(np.dot(K,R2), np.concatenate((np.eye(3), -C2), axis = 1))
    
    P = []
    reprojection_error = 0
    
    # Constructing the A matrix
    for i in range(pts1.shape[0]):
        
        A =  [pts1[i, 0] * camera_pose1[2, :] - camera_pose1[0, :],
              pts1[i, 1] * camera_pose1[2, :] - camera_pose1[1, :],
              pts2[i, 0] * camera_pose2[2, :] - camera_pose2[0, :],
              pts2[i, 1] * camera_pose2[2, :] - camera_pose2[1, :]]
        
        
        # Perform the singular value decomposition of the A matrix
        U, sigma, Vh = np.linalg.svd(A)
        
        # Extract the last row corresponding the nullspace of the linear equation
        p = Vh[-1,:]
        
        # Convert to homogenous coordinates
        w = p/p[3]
        
        # X,Y,Z co-ordinates in the world frame
        P.append(w)
        
        # Checking the Reprojection Error
        projection1 = np.dot(camera_pose1, w)
        projection2 = np.dot(camera_pose2, w)
        
        # Calculating the Reprojection error
        # Computing the eucledian distance between the projected image and the object in the world frame
        reprojection_error += np.linalg.norm(projection1[:2]/projection1[-1] - pts1[i])**2 + np.linalg.norm(projection2[:2]/projection2[-1] - pts2[i])**2
        
    return np.asarray(P), reprojection_error

def NonLinearTriangulationError(K, C1, R1, C2, R2, pts1, pts2, triangulated_points):
        
    """
    Inputs:
    K: The camera calibration matrix
    C1 and R1: The first camera pose
    C2 and R2: The second camera pose
    pts1 and pts2: The Nx2 point correspondence
    X0: The triangulated points (Nx4)
    """
    # The camera poses/ Projection Matrices
    P1 = np.dot(np.dot(K,R1), np.concatenate((np.eye(3), -C1), axis = 1))
    P2 = np.dot(np.dot(K,R2), np.concatenate((np.eye(3), -C2), axis = 1))
    
    # The Error terms
    u_reprojection1 = np.matmul(P1[0].reshape(1,4), triangulated_points) / np.matmul(P1[2].reshape(1,4), triangulated_points)
    v_reprojection1 = np.matmul(P1[1].reshape(1,4), triangulated_points) / np.matmul(P1[2].reshape(1,4), triangulated_points)
    u_reprojection2 = np.matmul(P2[0].reshape(1,4), triangulated_points) / np.matmul(P2[2].reshape(1,4), triangulated_points)
    v_reprojection2 = np.matmul(P2[1].reshape(1,4), triangulated_points) / np.matmul(P2[2].reshape(1,4), triangulated_points)
    
    diff1 = (pts1.T[0,:].reshape(1,4) - u_reprojection1)**2
    diff2 = (pts1.T[1,:].reshape(1,4) - v_reprojection1)**2
    diff3 = (pts2.T[0,:].reshape(1,4) - u_reprojection2)**2
    diff4 = (pts2.T[1,:].reshape(1,4) - v_reprojection2)**2
    
    reprojection_error = diff1 + diff2 + diff3 + diff4
    return reprojection_error
    
# Non Linear Triangulation
def NonLinearTriangulation(K, C1, R1, C2, R2, pts1, pts2):
    
    """
    Inputs:
    K: The camera calibration matrix
    C1 and R1: The first camera pose
    C2 and R2: The second camera pose
    pts1 and pts2: The Nx2 point correspondence
    
    Outputs:
    X: Nx3 Matrix that represents the triangulated points
    """
    # Get the Triangulated points array
    triangulated_points, reprojection_error = LinearTriangulation(K,C1,R1,C2,R2,pts1,pts2)
    
    # Reprojection Error Minimization using Levenberg-Marquardt Method
    args = (K, C1, R1, C2, R2, pts1, pts2)
    refined_triangulated, success = leastsq(NonLinearTriangulationError,triangulated_points,args = args)
    
    return refined_triangulated

# Check whether the point is infront of the camera pose or not
def DisambiguateCameraPose(Cset, Rset, Xset):
    
    """
    Inputs:
    Cset: 4 Camera Rotation Configurations
    Rset: 4 Camera rotation configurations
    Xset: 4 set of traingulated camera points
    
    Output:
    C,R: The best Camera Pose
    
    """
    best = 0
    
    for i in range(4):
        
        # Loop through each correspondence
        counter = 0
        for j in range(len(Xset[i])):
            
            # Cheirality Condition
            if(Rset[i][2,:]*(Xset[i][j,:] - Cset[i]) > 0):
                
                counter = counter + 1
                
        if counter > best:
            
            C = Cset[i]
            R = Rset[i]
            X0 = Xset[i]
            best = counter
    
    return C, R, X0

def LinearPnp(X,x,K):
    
    """
    Inputs:
    X: This is an Nx4 Homogenous Matrix whose row gives correspondence with the 2D Image
    x: This is an Nx2 Matrix whose row gives correspondence with the 3D Image
    K: The Camera Calibration Matrix
    
    Outputs:
    C, R: The Camera Pose
    """
    # Convert the 2D Correspondence to the optical world
    x = np.dot(np.linalg.inv(K), x.T)
    
    # Convert the 2d correspondence to homogenous coordinates
    x = np.concatenate((pts1,np.ones((pts1.shape[0],1))), axis=1)
    
    # Construct the A Matrix
    
    A = []
    
    for i in range(X.shape[0]):
        
        X_thilda = X[i,:]
        zeros = np.zeros((1,4))
        2d_correspondence = x[i,:]
        A_matrix = np.array([[zeros, -X_thilda, 2d_correspondence[1]*X_thilda],[X_thilda, zeros, -2d_correspondence[0]*X_thilda],[-2d_correspondence[1]*X_thilda, 2d_correspondence[0]*X_thilda, zeros]])
        A.append(A_matrix)
        
    # Stack the matrices to make an Nx12 Dimensional matrix
    A = np.vstack(A)
    
    # Perform the Singular Value Decomposition
    U, sigma, Vh = np.linalg.svd(A)
    
    # The last row of Vh corresponds to the solution
    # We reshape the solution into a 3x4 matrix
    P = Vh[-1,:].reshape(3,4)
    
    # The rotation Matrix is 
    R = P[:,:3]
    
    # The translation matrix is 
    t = P[:,3:]
    
    # Perform SVD as the least squares solution doesnot enforce orthogonality
    U, sigma, Vh = np.linalg.svd(R)
    
    # Enforce Orthogonality
    R = np.dot(U, Vh)
    
    # Camera Center 
    C = np.dot(-np.linalg.inv(R), t)
    
    # Some Constraints
    if np.linalg.det(R) < 0:
        
        R = -R
        C = -C
        
    return C, R
    
def NonLinearPnp(X,x,K,C,R):
    """
    Inputs:
    X: This is an Nx3 Matrix whose row gives correspondence with the 2D Image
    x: This is an Nx2 Matrix whose row gives correspondence with the 3D Image
    C, R: The Camera Pose
    
    Output:
    C, R: The Camera Pose
    """
    
    

In [42]:
K = np.array([[1,2,3],[4,5,6],[7,8,9]])
C1 = np.array([[1],[2],[3]])
R1 = np.array([[1,2,0],[0,-1,0],[0,0,1]])
C2 = np.array([[4],[5],[6]])
R2 = np.array([[1,2,3],[4,5,6],[7,8,9]])
pts1 = np.array([[1,2],[3,4],[5,6],[7,8]])
pts2 = np.array([[9,0],[11,12],[13,14],[15,16]])
triangulated_points, error = LinearTriangulation(K,C1,R1,C2,R2,pts1,pts2)
#NonLinearTriangulation(K,C1,R1,C2,R2,pts1,pts2)
#triangulated_points
pts1[0,:]
#np.dot(np.dot(K,R1), np.concatenate((np.eye(3), -C1), axis = 1))

array([1, 2])

In [65]:
A = np.array([[1],[2],[3],[4]])
K = np.array([[1,2,3],[4,5,6],[7,8,9]])
C1 = np.array([[1],[2],[3]])
R1 = np.array([[1,2,0],[0,-1,0],[0,0,1]])
C2 = np.array([[4],[5],[6]])
R2 = np.array([[1,2,3,4],[4,5,6,5],[7,8,9,6]])
pts1 = np.array([[1,2],[3,4],[5,6],[7,8]])
pts2 = np.array([[9,0],[11,12],[13,14],[15,16]])

#np.concatenate((pts1,np.ones((pts1.shape[0],1))), axis=1)
#R2[:,:3]
R2[:,3:]

array([[4],
       [5],
       [6]])

In [44]:
P = np.dot(np.dot(K,R1), np.concatenate((np.eye(3), -C1), axis = 1))
np.matmul(P[0,:].reshape(1,4),x)

array([[ 169.45325806, -174.75550062,  -45.91440416,   -6.        ]])

In [26]:
camera_model = 'C:\\Users\\shant\\dataset\\model\\'
image1 = 'C:\\Users\\shant\\dataset\\stereo\\centre\\1399381445704778.png'
k1, undistorted_image1 = intrinsic_matrix_undistorted_image(image1,camera_model)
image2 = 'C:\\Users\\shant\\dataset\\stereo\\centre\\1399381445767267.png'
k2, undistorted_image2 = intrinsic_matrix_undistorted_image(image2,camera_model)
pts1, pts2 = get_keypoints(undistorted_image1,undistorted_image2)

error: OpenCV(4.1.2) c:\projects\opencv-python\opencv\modules\imgproc\src\color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function '__cdecl cv::impl::`anonymous-namespace'::CvtHelper<struct cv::impl::`anonymous namespace'::Set<3,4,-1>,struct cv::impl::A0xe227985e::Set<1,-1,-1>,struct cv::impl::A0xe227985e::Set<0,2,5>,2>::CvtHelper(const class cv::_InputArray &,const class cv::_OutputArray &,int)'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 1


In [44]:
A = []
A.append(R1)
A.append(R1)
A = np.vstack(A)
A.shape

(6, 3)

In [64]:
x,y = ExtractCameraPose(K)
x

[array([[ 0.40824829],
        [-0.81649658],
        [ 0.40824829]]), array([[-0.40824829],
        [ 0.81649658],
        [-0.40824829]]), array([[ 0.40824829],
        [-0.81649658],
        [ 0.40824829]]), array([[-0.40824829],
        [ 0.81649658],
        [-0.40824829]])]